# Big Data Analytics
Praktikum Sommersemester 2023. <small>Version 1.1</small>

**Aufgabe 2: Abfragen mit Apache Drill** 

Machen Sie sich mit Apache Drill vertraut. Lösen Sie die Aufgaben in `sqlline` oder in einem Jupyter Notebook. Nutzen Sie die markierten Zellen im vorliegenden Notebook `BDA1_A2_Drill` für die Lösung und laden Sie es in Ilias hoch.

----

## Vorbereitung
* Erzeugen Sie den Ordner `work/drill-driver` im JupyterLab Workspace
* Legen Sie eine neue Datei `odbc.ini` dort an. Sie finden ein Beispiel unter `Big_Data_Analytics_1/public/drill-driver/`


In Drill sind mehrere Datenmengen konfiguriert und für Sie verwendbar:

* **dfs.data.\`co2data.tsv\`**<br>DFS Datenquelle in Form eines Datensatz an Sensordaten
* **dfs.bdapi.labels**<br>HTTP Datenquelle mit Sensornamen und -Positionen
* **dfs.bdapi.sensors**<br>HTTP Datenquelle mit einer fixierten Liste von JSON Objekten mit Sensordaten.
* **dfs.bdapi.sensorslastday**<br>HTTP Datenqelle mit einem JSON Objekt, das dynamische Sensordaten anbietet.
* **dfs.weather.sunrise**<br>HTTP Storage einer API, die Zeiten des Sonnenauf- und Sonnenuntergangs an einer gegebenen Position (fields lat und lon) zu einem Datum (field date) zurückgibt. Mehr Infos dazu auf der Webseite zur [sunrise-sunset.org/api](https://sunrise-sunset.org/api).

## Aufgabe 2 a

Verschaffen Sie sich einen Überblick über die Sensordaten, die Sie im Drill unter ``dfs.data.`co2data.tsv` `` finden. Beantworten Sie die folgenden
Fragen, indem Sie jeweils eine SQL-Query gegen das Drill Cluster ausführen:
    
1. Wieviele verschiedene Sensoren (angegeben im Feld _source_) enhält die Datenmenge?
2. Wieviele Datenpunkte je Sensor liegen vor? Geben Sie sowohl Sensor als auch Anzahl aus.
3. Bereiten Sie in einer SQL-Query die Werte in einzelnen Spalten so vor, dass sie sinnvolle Datentypen aufweisen:
    1. Sowohl _humidity_, _temperature_ als auch _co2_ sollen als auf zwei Nachkommastellen gerundete Fließkommazahlen verfügbar sein.
    2. Die erste Spalte gibt den Zeitstempel als Unix Epoch mit Mikrosekunden an. Machen Sie daraus einen Drill Timestamp
4. Was ist der höchste, und was der niedrigste Temperaturwert? Beide Werte sollen zusammen in einer Query angefragt werden. Kennzeichnen Sie die beiden Felder mit einem sprechenden Namen.
5. Was ist der durchschnittliche CO<sub>2</sub>-Wert je Sensor in der Datenmenge?


## Erklärung
Folgendes ist ein `Programm`, der die Verbindung auf die `ODBC` Datenbank erstellt, SQL Queries entsprechend der 
Aufgabenstellung ausführt und und auf dem Notebook ausgibt.
----

In [2]:
import pyodbc
import pandas as pd

Die Funktion `group_tuples_by_index` nimmt eine Liste von N Tupeln, erstellt eine andere Liste mit N leeren Listen und füllt diese Listen dann
mit Werten mit dem gleichen Index aus den Tupeln

In [3]:
def group_tuples_by_index(tuples_list):
    grouped_lists = [[] for _ in range(len(tuples_list[0]))]
    
    for tpl in tuples_list:
        for i, value in enumerate(tpl):
            grouped_lists[i].append(value)
    
    return grouped_lists

In diesem Program verbinen wir uns mit Drill wie es in der Vorlesung vorgestellt wurde und führen meherere Programme laut der Aufgabenstellung aus. <br>
In unserem ersten SQL Query haben wir erst die Tabelle `dfs.data.co2data.tsv` zugegriffen und den Subquery erstellt, wo wir den Header mit `offset 1` entfernen und dann in Query die Anzahl der einzigartigen Sensoren aus diesem Subquery berechnen. 

In [4]:
# Aufgabe 2a: 1. 
def connect(dsn='drill'):
    """opens the connection to given DSN"""
    conn = pyodbc.connect("DSN="+dsn, autocommit=True)
    if not conn.closed:
        print(f"connected to drillbit {dsn}")
        return conn
    else:
        print(f"could not connect to {dsn}")
        return None

con = connect()
query = '''select count(distinct tab.sensor) as unique_sensors from (select columns[1] as sensor from dfs.data.`co2data.tsv` offset 1) as tab'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchone()
print("Amount of unique sensors: " + str(res[0]))

connected to drillbit drill
Amount of unique sensors: 22


### Tabellarische Darstellung
Hier und weiter stellen wir die Daten, die wir mit SQL Queries erhalten haben, tabellarisch mithilfe
der Bibliothek `pandas` dar. Wir benutzen Dataframe, wo wir als Parameter eine Dictionary eingeben, wo der Key als Tabellenheader und
Value als Liste der Werte dargestellt ist.
----

Hier und weiter gehen wir mit dem Header nämlich gleich vor, wo wir den Header entfernen, indem wir einen Subquery erstellen. Dann stellen wir
die einzelnen Sensoren und ihre Anzahl dar, die durch `count` berechnet wurde. Dann haben wir die Liste der Tupeln genommen und in die
Funktion `group_tuples_by_index` übergeben und als Rückgabe eine Liste erhalten, die wir als Spaltenwerte verwenden können. Schließlich
haben wir eine Dictionary mit unseren Werten erstellt und als ein Tabelle dargestellt.

In [28]:
# Aufgabe 2a: 2.
query = '''select tab.sensor, count(tab.sensor) as amount from (select columns[1] as sensor from dfs.data.`co2data.tsv` offset 1) as tab group by sensor'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
result_list = group_tuples_by_index(res)
data = {'sensors': result_list[0], 'amount': result_list[1]}
df = pd.DataFrame(data)
df

,sensors,amount
0,s_d8bfc0147061_,1
1,s_8caab57cc961_284337,1131861
2,s_e8db84c5f33d_,13
3,s_8caab57cc961_,7
4,s_10521c01cf19_,2
5,s_3c6105d3abae_,2
6,s_e8db84c5f33d_0xdeadbeef,1515
7,s_8caab57c3e19_282028,1561045
8,s_e8db84c5f33d,83
9,s_8caab57a6dd9_288065,2781611


Hier haben wir die `co2`, `temperature`, `humidity` und `timestamp` Werte erst zu entsprechenden Datentypen gecastet und dann 
auf 2 Nachkommastellen gerundet und wie bei vorigen Aufgaben in der Tabelle dargestellt.

In [29]:
# Aufgabe 2a: 3.
query = '''select round(cast(columns[3] as float), 2) as co2, round(cast(columns[4] as float), 2) as temperature, round(cast(columns[5] as float), 2) as humidity, to_timestamp(cast(columns[6] as double)) as drillTime from dfs.data.`co2data.tsv` offset 1'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
result_list = group_tuples_by_index(res)
data = {'co2': result_list[0], 'temperature': result_list[1], 'humidity': result_list[2], 'drill_time': result_list[3]}
df = pd.DataFrame(data)
df

,co2,temperature,humidity,drill_time
0,420.0,23.0,36.0,2021-04-01 12:08:48
1,421.0,24.0,32.0,2021-04-01 12:08:56
2,420.0,24.0,32.0,2021-04-01 12:08:59
3,651.0,20.0,44.0,2021-04-01 12:09:21
4,422.0,23.0,36.0,2021-04-01 12:09:29
...,...,...,...,...
24995,773.0,20.0,49.0,2021-04-01 12:07:47
24996,773.0,20.0,49.0,2021-04-01 12:07:49
24997,773.0,20.0,49.0,2021-04-01 12:07:54
24998,422.0,23.0,36.0,2021-04-01 12:08:05


Hier bereiten wir unsere Daten wieder in Subquery mit dem Entfernen vom Header vor und sagen, dass die Werte nicht Null sein sollen. Danach
nehmen wir die fertigen Daten, casten sie und berechnen durch die Funktionen `min` und `max` die Mindest- und Höchstwerte.

In [30]:
# Aufgabe 2a: 4.
query = '''select min(cast(tab.temp as float)) as min_temp, max(cast(tab.temp as float)) as max_temp from (select columns[4] as temp from dfs.data.`co2data.tsv` where columns[4] <> 'null' offset 1) as tab'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
result_list = group_tuples_by_index(res)
data = {'min_temperature': result_list[0], 'max_temperature': result_list[1]}
df = pd.DataFrame(data)
df

,min_temperature,max_temperature
0,-1.0,36.0


In dieser Aufgabe haben wir alle einzigartige Sensoren genommen und die durchschnittliche CO2 Werte je Sensor berechnet, wobei 
wir Subquery mit Sensoren und CO2 Werte erstellt haben, wo wir keine Null Werte nehmen und Header entfernen, danach haben wir
die CO2 Werte erst auf `float` gecastet und dann die Werte dargestellt.

In [34]:
# Aufgabe 2a: 5.
query = '''select distinct tab.sensor, avg(cast(tab.co2 as float)) as average_co2 from (select columns[1] as sensor, columns[3] as co2 from dfs.data.`co2data.tsv` where columns[3] <> 'null' offset 1) as tab group by sensor'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
result_list = group_tuples_by_index(res)
data = {'sensor': result_list[0], 'avg_co2': result_list[1]}
df = pd.DataFrame(data)
df

,sensor,avg_co2
0,s_10521c0202ab_284839,436.763566
1,s_e8db84c5f33d,1403.228916
2,s_8caab57c3e19_282028,876.931584
3,s_e8db84c5f33d_281913,531.210462
4,s_e8db84c5f771_300390,954.087763
5,s_e8db84c5f33d_0xdeadbeef,-1.000000
6,s_3c6105d3abae_299589,833.976671
7,s_8caab57a6dd9_288065,654.238816
8,s_e8db84c5f771_,863.000000
9,s_10521c01cf19_262520,500.677816


## Aufgabe 2 b

1. Verknüpfen Sie die Daten aus ``dfs.data.`co2data.tsv` ``  mit den Daten aus `bdapi.labels`. Geben Sie, wenn möglich, den Sensornamen ( _name_ ) und Besitzer ( _owner_ ) zu jeder Seriennummer aus. Falls Daten fehlen, geben Sie die Seriennummer dennoch aus.<br>Hinweis: Die Seriennummern liegen bei den beiden Datenmengen in unterschiedlicher Repräsentation vor. Untersuchen Sie die Datenmengen und ändern Sie Ihre Query so ab, dass die Seriennummern korrekt verknüpft werden. 
2. Überführen Sie diese Query (aus 2b) 1) in eine View. Sie haben Schreibrecht im Workspace `dfs.tmp`. Legen Sie darin eine View namens `labels_<ihre Ilias-ID>` an.<br>Beispiel: Lautet Ihre Ilias-ID `mr1337s` nutzen Sie `labels_mr1337s`. Falls diese View bereits existiert soll sie überschrieben werden!
3. Nutzen Sie Ihre neu angelegte View nun und zeigen Sie alle Sensoren des Besitzers `Elsen` an.
4. Wann ein Sensor eines unbekannten Besitzers (`unknown`) zum ersten Mal gesendet?
5. Lassen Sie sich den physischen Plan für die Query aus 2b 4. anzeigen. 

---------------------------------
Im folgenden Query verknüpfen wir die Daten aus ``dfs.data.`co2data.tsv` ``  mit den Daten aus `bdapi.labels`. Wir haben hier
ein Subquery mit dem Entfernen des Headers und Null Werten erstellt, daraus dann durch `substring` Funktion die Seriennummer 
abgeschnitten, sodass wir den gleichen Seriennummer String wie in der Datenmenge `bdapi.labels` haben. Danach haben wir mithilfe diesen
gleichen String Werten die Tabellen durch `left join` verbunden, weil laut der Aufgabenstellung wir alle Seriennummer ausgeben sollen.
Zum Schluss geben wir aus dieser durch verknüpfung erstellte Tabelle den vollen Seriennummer, Seriennummer, Sensornamen und den 
Besitzer des Sensors.

In [7]:
# Aufgabe 2b: 1.
query = '''select distinct col.fullSerNum, col.serNum, api.name, api.owner from (select columns[1] as fullSerNum, upper(substring(columns[1], 3, 12)) as serNum from (select * from dfs.data.`co2data.tsv` where columns[1] <> 'null' offset 1)) as col left join bdapi.labels as api on col.serNum = api.serial order by col.serNum'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
result_list = group_tuples_by_index(res)
data = {'full_ser_num': result_list[0], 'ser_num': result_list[1], 'name': result_list[2], 'owner': result_list[3]}
df = pd.DataFrame(data)
df

,full_ser_num,ser_num,name,owner
0,s_10521c01cf19_262520,10521C01CF19,Admin S4,Remmy
1,s_10521c01cf19_,10521C01CF19,Admin S4,Remmy
2,s_10521c0202ab_284839,10521C0202AB,Main Station S4,Elsen
3,s_3c6105d3abae_,3C6105D3ABAE,Main Station S5,Elsen
4,s_3c6105d3abae_299589,3C6105D3ABAE,Main Station S5,Elsen
5,s_8caab57a6dd9,8CAAB57A6DD9,Admin S3,Remmy
6,s_8caab57a6dd9_0xdeadbeef,8CAAB57A6DD9,Admin S3,Remmy
7,s_8caab57a6dd9_288065,8CAAB57A6DD9,Admin S3,Remmy
8,s_8caab57a6dd9_,8CAAB57A6DD9,Admin S3,Remmy
9,s_8caab57c3e19_282028,8CAAB57C3E19,Main Station S2,Elsen


Die Query aus der Aufgabe 2b.1 überführen wir in eine View im Workspace `dfs.tmp`. Dafür benutzen wir `create or replace view` für den Fall, dass wir
die View mit einer anderen ersetzen wollen.

In [17]:
# Aufgabe 2b: 2.
query = '''create or replace view dfs.tmp.labels_nt2790s as select distinct col.fullSerNum, col.serNum, api.name, api.owner from (select columns[1] as fullSerNum, upper(substring(columns[1], 3, 12)) as serNum from (select * from dfs.data.`co2data.tsv` where columns[1] <> 'null' offset 1)) as col left join bdapi.labels as api on col.serNum = api.serial order by col.serNum'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

(True, "View 'labels_nt2790s' replaced successfully in 'dfs.tmp' schema")


Folgender Query zeigt alle Sensoren, wo der Besitzer = 'Elsen' ist. Das haben wir aus der erstellten View aus der `dfs.tmp.labels_nt2790s`
abgefragt und dargestellt.

In [6]:
# Aufgabe 2b: 3.
query = '''select * from dfs.tmp.`labels_nt2790s` where owner = 'Elsen' '''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
result_list = group_tuples_by_index(res)
data = {'full_ser_num': result_list[0], 'ser_num': result_list[1], 'name': result_list[2], 'owner': result_list[3]}
df = pd.DataFrame(data)
df

,full_ser_num,ser_num,name,owner
0,s_10521c0202ab_284839,10521C0202AB,Main Station S4,Elsen
1,s_3c6105d3abae_299589,3C6105D3ABAE,Main Station S5,Elsen
2,s_3c6105d3abae_,3C6105D3ABAE,Main Station S5,Elsen
3,s_8caab57c3e19_282028,8CAAB57C3E19,Main Station S2,Elsen
4,s_8caab57c3e19_,8CAAB57C3E19,Main Station S2,Elsen
5,s_8caab57cc961_,8CAAB57CC961,Main Station S3,Elsen
6,s_8caab57cc961_284337,8CAAB57CC961,Main Station S3,Elsen
7,s_d8bfc014724e_262793,D8BFC014724E,Main Station S1,Elsen


In dieser Aufgabe haben wir verschachtelte Queries. Die 1. Subquery dient zum Entfernen des Headers und Null Werten, 
die 2. Subquery schneidet die Seriennummer so, dass wir diese mit den Seriennummern aus der `bdapi.labels` verbinden. Darüber hinaus
nehmen wir die Zeitwerte in der 2. Subquery. Das resultierende aus der 2. Subquery verknüpfen wir mit der Tabelle aus `bdapi.labels`
mithilfe der Seriennummern mit `left join` damit wir keine Seriennummer verlieren. Zum Schluss casten, umwandeln auf `timestamp` und suchen
einen minimalen Wert des Zeitwertes, wobei der Besitzer `Unknown` ist.

In [8]:
# Aufgabe 2b: 4.
query = '''select min(to_timestamp(cast(col.first_time as double))) as first_time, api.owner from (select upper(substring(columns[1], 3, 12)) as serNum, columns[6] as first_time from (select * from dfs.data.`co2data.tsv` where columns[1] <> 'null' offset 1)) as col left join bdapi.labels as api on col.serNum = api.serial where api.owner = 'Unknown' group by api.owner'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
result_list = group_tuples_by_index(res)
data = {'first_time': result_list[0], 'owner': result_list[1]}
df = pd.DataFrame(data)
df

,first_time,owner
0,2021-03-30 11:50:05,Unknown


Im Folgenden zeigen wir mithilfe des `explain plan for` den physischen Plan für 2b.4

In [9]:
# Aufgabe 2b: 5.
query = '''explain plan for select min(to_timestamp(cast(col.first_time as double))) as first_time, api.owner from (select upper(substring(columns[1], 3, 12)) as serNum, columns[6] as first_time from (select * from dfs.data.`co2data.tsv` where columns[1] <> 'null' offset 1)) as col left join bdapi.labels as api on col.serNum = api.serial where api.owner = 'Unknown' group by api.owner'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
for row in res:
    print(row)

('00-00    Screen\n00-01      Project(first_time=[$0], owner=[$1])\n00-02        ComplexToJson\n00-03          ProjectAllowDup(first_time=[$0], owner=[$1])\n00-04            UnionExchange\n01-01              Project(first_time=[$1], owner=[$0])\n01-02                HashAgg(group=[{0}], first_time=[MIN($1)])\n01-03                  Project(owner=[$0], first_time=[$1])\n01-04                    HashToRandomExchange(dist0=[[$0]])\n02-01                      UnorderedMuxExchange\n03-01                        Project(owner=[$0], first_time=[$1], E_X_P_R_H_A_S_H_F_I_E_L_D=[hash32AsDouble($0, 1301011:BIGINT)])\n03-02                          HashAgg(group=[{0}], first_time=[MIN($1)])\n03-03                            Project(owner=[$4], $f1=[TO_TIMESTAMP(CAST($1):DOUBLE)])\n03-04                              HashJoin(condition=[=($0, $3)], joinType=[inner], semi-join: =[false])\n03-06                                Project(serNum=[UPPER(SUBSTRING(ITEM(ITEM($0, \'columns\'), 1), 3, 12))], fir

## Aufgabe 2 c

1. Die Datenmenge unter `dfs.bdapi.sensorslastday` ändert sich in Intervallen.  Finden Sie einen Weg, diese Datenmenge nutzbar zu machen. (Tipp: Drill bietet Funktionen für komplexe Datentypen an, z.B. [FLATTEN()](https://drill.apache.org/docs/flatten/)). Die Antwort Ihrer Query sollte die Felder `timewindow`, `celsius` und `humidity` liefern.
2. Geben Sie mit einer einzigen Query die beiden Datenpunkte der extremen Temperaturen (`celsius`) aus, d.h. die Zeile mit der höchsten Temeperatur und die Zeile mit der niedrigsten Temperatur.
3. Optional: Finden Sie für die Datenmenge `dfs.bdapi.sensorslastday` zu jedem Zeitraum heraus, ob es Tag ist oder Nacht. Das können Sie mithilfe der Datenmenge `dfs.weather.sunrise` tun. Nehmen Sie für den Sensor in der Datenmenge die Geokoordinaten `50.75410842895508`, `6.08587121963501` an.  

-------------------------------------------
In dieser Aufgabe bekommen wir erst einen Subquery aus der Liste der Sensorwerte, die sich immer wieder ändert. Das machen wir
mithilfe von `flatten` und dann zeigen wir einfach die angefragte Werte `timewindow`, `celsius`, `humidity`.

In [11]:
# Aufgabe 2c: 1.
query = '''select t.item.timewindow as timewindow, t.item.celsius as celsius, t.item.humidity as humidity from (select flatten(resultset) as item from bdapi.sensorslastday) as t'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
result_list = group_tuples_by_index(res)
data = {'timewindow': result_list[0], 'celsius': result_list[1], 'humidity': result_list[2]}
df = pd.DataFrame(data)
df

,timewindow,celsius,humidity
0,2023-05-25 01:45:00,20.456966,38.374599
1,2023-05-25 01:45:00,23.290625,32.889248
2,2023-05-25 01:30:00,20.513476,38.423222
3,2023-05-25 01:30:00,23.334244,32.815880
4,2023-05-25 01:15:00,23.381315,32.764516
...,...,...,...
995,2023-05-19 21:30:00,22.531510,34.808317
996,2023-05-19 21:15:00,22.643034,34.795190
997,2023-05-19 21:15:00,25.873959,30.480114
998,2023-05-19 21:00:00,22.742480,34.402058


In der letzten Aufgabe gehen wir gleich wie in der 1. Aufgabe vor und bereiten unsere Werte mit `flatten()` vor. Dananch
suchen wir die minimalen und maximalen Werte mit `min()` und `max()`.

In [12]:
# Aufgabe 2c: 2.
query = '''select min(t.item.celsius) as min_celsius, max(t.item.celsius) as max_celsius from (select flatten(resultset) as item from bdapi.sensorslastday) as t'''
csr = con.cursor()
csr.execute(query)
res = csr.fetchall()
result_list = group_tuples_by_index(res)
data = {'min_celsius': result_list[0], 'max_celsius': result_list[1]}
df = pd.DataFrame(data)
df

,min_celsius,max_celsius
0,-34.479685,27.963542


In [ ]:
# Aufabe 2c: 3. optional

_____